Calibration is the process of finding the best values of the parameters of a financial model that minimize the differences between the model prices and market prices of a set of financial instruments. In other words, calibration is used to ensure that the model is consistent with the observed market data.



In the case of interest rate models, calibration is important because interest rates are fundamental variables that affect the pricing and risk management of a wide range of financial instruments, such as bonds, swaps, options, and other derivatives. Moreover, interest rate markets are highly liquid and traded by many market participants, so the market prices of interest rate instruments are a good reflection of market expectations and perceptions of future interest rates.

In [1]:
 from QuantLib import *

In [2]:
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from collections import namedtuple
import math

In [4]:
today = Date(18, April, 2023);
settlement= Date(19, April, 2024);
Settings.instance().evaluationDate = today;
term_structure = YieldTermStructureHandle(
FlatForward(settlement,0.04875825,Actual365Fixed())
)
index = Euribor1Y(term_structure)


In [5]:
CalibrationData = namedtuple("CalibrationData",
"start, length, volatility")
data = [CalibrationData(1, 5, 0.1148),
CalibrationData(2, 4, 0.1108),
CalibrationData(3, 3, 0.1070),
CalibrationData(4, 2, 0.1021),
CalibrationData(5, 1, 0.1000 )]


In [6]:
def create_swaption_helpers(data, index, term_structure, engine):
  swaptions = []
  fixed_leg_tenor = Period(1, Years)
  fixed_leg_daycounter = Actual360()
  floating_leg_daycounter = Actual360()
  for d in data:
    vol_handle = QuoteHandle(SimpleQuote(d.volatility))
    helper = SwaptionHelper(Period(d.start, Years),
    Period(d.length, Years),
    vol_handle,
    index,
    fixed_leg_tenor,
    fixed_leg_daycounter,
    floating_leg_daycounter,
    term_structure
    )
    helper.setPricingEngine(engine)
    swaptions.append(helper)
  return swaptions

In [7]:
def calibration_report(swaptions, data):
  columns = ["Model Price", "Market Price", "Implied Vol", "Market Vol", "Rel Er\
  ror Price", "Rel Error Vols"]
  report_data = []
  cum_err = 0.0
  cum_err2 = 0.0
  for i, s in enumerate(swaptions):
    model_price = s.modelValue()
    market_vol = data[i].volatility
    black_price = s.blackPrice(market_vol)
    rel_error = model_price/black_price - 1.0
    implied_vol = s.impliedVolatility(model_price,
    1e-5, 50, 0.0, 0.50)
    rel_error2 = implied_vol/market_vol-1.0
    cum_err += rel_error*rel_error
    cum_err2 += rel_error2*rel_error2
    report_data.append((model_price, black_price, implied_vol,
    market_vol, rel_error, rel_error2))
  print ("Cumulative Error Price: %7.5f" % math.sqrt(cum_err))
  print( "Cumulative Error Vols : %7.5f" % math.sqrt(cum_err2))
  return DataFrame(report_data,columns= columns, index=['']*len(report_data))


In [8]:
model = HullWhite(term_structure);
engine = JamshidianSwaptionEngine(model)
swaptions = create_swaption_helpers(data, index, term_structure, engine)
optimization_method = LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)
a, sigma = model.params()
print ("a = %6.5f, sigma = %6.5f" % (a, sigma))


a = 0.05247, sigma = 0.00688


In [9]:
calibration_report(swaptions, data)

Cumulative Error Price: 0.29531
Cumulative Error Vols : 0.29616


,Model Price,Market Price,Implied Vol,Market Vol,Rel Er ror Price,Rel Error Vols
,0.010269,0.013374,0.088110,0.1148,-0.232146,-0.232494
,0.011277,0.012314,0.101447,0.1108,-0.084188,-0.084416
,0.010099,0.010047,0.107550,0.1070,0.005122,0.005142
,0.007585,0.006969,0.111170,0.1021,0.088394,0.088836
,0.004128,0.003635,0.113644,0.1000,0.135601,0.136439


The code above is an implementation of the Hull-White one-factor model to calibrate a term structure of interest rates to a set of swaption volatilities. The model is used to calculate the price of European swaptions and compare them to market prices, in order to find the best values of the model parameters that minimize the differences between model and market prices.

The main steps of the implementation are:

Set the evaluation date, settlement date, and the term structure of interest rates.


Define the calibration data, which consists of the start time, length, and volatility of each swaption in the set.



Define the Euribor index and create the swaption helpers, which are objects that provide the necessary information for pricing the swaptions.
Define the calibration function, which calculates the model prices, implied volatilities, and errors between model and market prices for each swaption.



Define the Hull-White model and the Jamshidian swaption engine, which are used to calculate the model prices.
Define the optimization method and end criteria, which are used to find the best model parameters that minimize the errors between model and market prices.



Calibrate the model to the swaption set using the model.calibrate function.
Print the calibrated values of the model parameters and the calibration report, which shows the errors between model and market prices and implied volatilities for each swaption, as well as cumulative errors.






The implementation uses the QuantLib library, which provides a set of tools for pricing and modeling financial instruments. The Hull-White model is a one-factor model that assumes that the short-term interest rates follow a mean-reverting process with constant mean and volatility. The Jamshidian swaption engine is a numerical method for pricing swaptions. The Levenberg-Marquardt optimization method is used to minimize the sum of squared errors between model and market prices. The end criteria specify when the optimization should stop if convergence is not achieved.